In [1]:
# -*- coding: utf-8 -*-
import lightgbm as lgb
import csv
import pandas as pd
import numpy as np
import time
import json
from joblib import load, dump
from tqdm import tqdm
import re
from utils.features_ents import feature_ents
from utils.ner import ner

In [2]:
class Test():
    def __init__(self):
        #测试文件
        self.test_file = 'data/coreEntityEmotion_test_stage1.txt'
        #训练好的模型地址
        self.ents_model = load('models/model1.joblib')
        
    def test(self):
        test_file = open(self.test_file)
        process_num = "1"
        res_file = open("./results/result_"+str(process_num)+".txt",'w', newline='')
        write = csv.writer(res_file)
        fea_ents = feature_ents()
        
        count = 0
        process_news = []
        for news in tqdm(test_file):
            count += 1
            if count >= 10:   #测试能否跑通，测试完去掉
                break
            if count >= 0:
                news = json.loads(news)
                ent_fea = fea_ents.combine_features(news)
                ents = []
                emos = []
                #预测实体
                ent_predict_result = []
                for fea in ent_fea:
                    ent_score = self.ents_model.predict([fea[1]])
                    ent_predict_result.append([fea[0][0],ent_score])

                ent_predict_result.sort(key=lambda x: x[1], reverse = True)
            
                #选前三个实体
                c = 0
                for pred in ent_predict_result:
                    if c == 3:
                        break
                    ents.append(pred[0])
                    c += 1
                    
#                 情感预测 :)
                for ent in ents:
                    emos.append("POS")

#                 print('{}\t{}\t{}'.format(news['newsId'], ','.join(ents), ','.join(emos)))
                row = ['{}\t{}\t{}'.format(news['newsId'], ','.join(ents), ','.join(emos))]
                write.writerow(row)
        print("done")

            
if __name__ == '__main__':
    test = Test()
    test.test()

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.387 seconds.
Prefix dict has been built succesfully.
9it [00:06,  1.61it/s]


done
